# Part I (Analytical Option Formulae)


Consider the following European options:\
• Vanilla call/put\
• Digital cash-or-nothing call/put\
• Digital asset-or-nothing call/put\
\
Derive and implement the following models to value these options in Python:\
1 Black-Scholes model\
2 Bachelier model\
3 Black model\
4 Displaced-diffusion model

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import norm

from google.colab import drive
drive.mount('/content/drive/')

import os
os.getcwd()
os.chdir('/content/drive/My Drive/Rat/QF620 Stochastic')

Mounted at /content/drive/


### 1. Black-Scholes Model(European):
\
  Vanilla Call/Put:

- Call: $C_{BS} = S_0Φ(d_1) - Ke^{-rT}Φ(d_2)$
- Put: $P_{BS} = Ke^{-rT}Φ(-d_2) - S_0Φ(-d_1)$\
\
  Where \
$d_1 = \frac{\log\frac{S_0}{K} + \left(r + \frac{\sigma^2}{2}\right)T}{\sigma\sqrt{T}}$ \
$d_2 = d_1 - \sigma\sqrt{T}$\
Note: "log" is log normal "ln" \
\
Variables:\
Φ = cumulative distribution function (CDF)\
$d_1$: "moneyness" of the option that determines the probability that option will be ITM \
$d_2$: adjusts the stdv. of $d_1$ of "intrinsic probability" that the option will expire ITM. \
\
$S_0$: initial stock price,\
$K$: strike price of the option.\
$T$: the expiry time (annual), \
$r$: risk-free interest rate (annual),\
$\sigma$: volatility of the underlying asset's returns (annual standard deviation) \



In [ ]:
# Vanilla call/put:
def black_scholes_vanilla(S, K, T, r, sigma, option_type):

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return price



The digital cash-or-nothing option pays a fixed amount of cash if the option finishes in the money at expiration, regardless of the underlying asset's price. The payout of digital cash-or-nothing is $Q$ \
- For a digital cash-or-nothing $call$, the payout is $Q$ if $S_T$ > $K$, and 0 otherwise. \
$C_{digital-cash}$ = $Q$ $e^{-rT}$Φ($d_2$) \
as we need $ C_{digital-cash}  = Q e^{-rT} \mathbb{Q} \left[ S_T > K\right]$ \

 $C_{digital-cash}=e^{-rT}\mathbb{E}\left[\mathbb{Q}_{S_T>K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty e^{-\frac{x^2}{2}}\,dx=e^{-rT}\Phi(d_2)$. \

- For a digital cash-or-nothing $put$, the payout if $Q$ if $S_T$ < K, and 0 otherwise. \
$P_{digital-cash}$ = $P$ $e^{-rT}$Φ($-d_2$) \
as we need $ P_{digital-cash}  = Q e^{-rT} \mathbb{Q} \left[ S_T < K\right]$ \

 $P_{digital-cash}=e^{-rT}\mathbb{E}\left[\mathbb{Q}_{S_T<K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}e^{-\frac{x^2}{2}}\,dx=e^{-rT}\Phi(-d_2)$. \

where $d_2 = \frac{\log(\frac{S_0}{K}) + \left(r - \frac{\sigma^2}{2}\right)T}{\sigma\sqrt{T}}$ \





In [ ]:
# digital cash-or-nothing Call/Put
def black_scholes_digital_cash(S, K, T, r, sigma, option_type):

    d2 = (np.log(S / K) + (r - 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        price = np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = np.exp(-r * T) * norm.cdf(-d2)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return price


The digital asset-or-nothing option pays the underlying asset's price $S_T$ if the option finishes in the money at expiration, regardless of the strike price. The payout of digital asset-or-nothing is $S_T$.

- For a digital asset-or-nothing $call$, the payout is $S_T$ if $S_T > K$, and 0 otherwise. \
$C_{digital-asset} = S_0 \Phi(d_1)$ \
as we need $ C_{digital-asset} = e^{-rT} \mathbb{E} \left[ S_T \cdot \mathbf{1}(S_T > K) \right] $ \

 $ C_{digital-asset} = \frac{e^{-rT}}{\sqrt{2 \pi}} \int_{x^*}^{\infty} S_0 e^{\left( r - \frac{\sigma^2}{2} \right) T + \sigma \sqrt{T} x} e^{-\frac{x^2}{2}} \, dx = S_0 \Phi(d_1) $

- For a digital asset-or-nothing $put$, the payout is $S_T$ if $S_T < K$, and 0 otherwise. \
$P_{digital-asset} = S_0 \Phi(-d_1)$ \
as we need $ P_{digital-asset} = e^{-rT} \mathbb{E} \left[ S_T \cdot \mathbf{1}(S_T < K) \right] $ \

 $ P_{digital-asset} = \frac{e^{-rT}}{\sqrt{2 \pi}} \int_{-\infty}^{x^*} S_0 e^{\left( r - \frac{\sigma^2}{2} \right) T + \sigma \sqrt{T} x} e^{-\frac{x^2}{2}} \, dx = S_0 \Phi(-d_1) $

where $ d_1 = \frac{\log\left(\frac{S_0}{K}\right) + \left(r + \frac{\sigma^2}{2}\right) T}{\sigma \sqrt{T}} $.


In [ ]:
# Digital asset-or-nothing Call/Put
def black_scholes_digital_asset(S, K, T, r, sigma, option_type):

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        price = S * norm.cdf(d1)
    elif option_type == 'put':
        price = S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return price


### 2. Bachelier Model (European):
Vanilla Call/Put:
- Call: $C_{Bachelier} = e^{-rT}\left[(S_0 - K)\Phi(d) + \sigma\sqrt{T}\phi(d)\right]$, as $x > \frac{S_0 - K}{\sigma\sqrt{T}} = x^* = d$

- Put: $P_{Bachelier} = e^{-rT}\left[(K - S_0)\Phi(-d) + \sigma\sqrt{T}\phi(-d)\right]$, as $x < \frac{S_0 - K}{\sigma\sqrt{T}} = x^* = -d$

 where $d = \frac{S_0 - K}{\sigma\sqrt{T}}$ \

 \

 Variables: \
  $\Phi$ is the cumulative distribution function (CDF) of the standard normal   distribution, \
  $\phi$ is the probability density function (PDF) of the standard normal   distribution. \
  $d$ is the input to these standard normal functions to determine the probability of moneyness and expected payoff. \
  \
  $S_0$ initial stock price, \
  $K$ strike price of the option, \
  $T$ the expiry time (annual), \
  $r$ risk-free interest rate (annual), \
  $\sigma$ volatility of the underlying asset's returns (annual standard deviation), \


In [ ]:
import numpy as np
from scipy.stats import norm

def N(x):
    """Cumulative distribution function for standard normal."""
    return norm.cdf(x)

def n(x):
    """Probability density function for standard normal."""
    return norm.pdf(x)


In [ ]:
# Bachelier Vanilla Call/Put:
def bachelier_vanilla_call(S0, K, T, sigma, r):
    d = (S0 - K) / (sigma * np.sqrt(T))
    return np.exp(-r * T) * ((S0 - K) * N(d) + sigma * np.sqrt(T) * n(d))


def bachelier_vanilla_put(S0, K, T, sigma, r):
    d = (S0 - K) / (sigma * np.sqrt(T))
    return np.exp(-r * T) * ((K - S0) * N(-d) + sigma * np.sqrt(T) * n(d))


The digital cash-or-nothing option pays a fixed amount of cash if the option finishes in the money at expiration, regardless of the underlying asset's price. The payout of digital cash-or-nothing is $Q$ \
 \
- For a digital cash-or-nothing $call$, the payout is $Q$ if $S_T$ > $K$, and 0 otherwise. \
$C_{digital-cash}$ = $Q$ $e^{-rT}$Φ($d$) \
as we need $ C_{digital-cash}  = Q e^{-rT} \mathbb{Q} \left[ S_T > K\right]$ \

 $C_{digital-cash}=e^{-rT}\mathbb{E}\left[\mathbb{Q}_{S_T>K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty e^{-\frac{x^2}{2}}\,dx=e^{-rT}\Phi(d)$. \

- For a digital cash-or-nothing $put$, the payout if $Q$ if $S_T$ < K, and 0 otherwise. \
$P_{digital-cash}$ = $P$ $e^{-rT}$Φ($-d$) \
as we need $ P_{digital-cash}  = Q e^{-rT} \mathbb{Q} \left[ S_T < K\right]$ \

 $P_{digital-cash}=e^{-rT}\mathbb{E}\left[\mathbb{Q}_{S_T<K}\right]=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{x^*}e^{-\frac{x^2}{2}}\,dx=e^{-rT}\Phi(-d)$. \

where $d = \frac{S_0 - K}{\sigma\sqrt{T}}$ \


In [ ]:
# Bachelier digital cash-or-nothing Call/Put:
def bachelier_digital_cash_call(S0, K, T, sigma, Q=1.0): # Assume our pay off is $1
    d = (S0 - K)/(sigma * np.sqrt(T))
    return np.exp(-r * T) * norm.cdf(d)

def bachelier_digital_cash_put(S0, K, T, sigma, Q=1.0):
    d = (S0 - K)/(sigma * np.sqrt(T))
    return np.exp(-r * T) * norm.cdf(-d)


The Bachelier model assumes that the underlying asset price follows a normal distribution. For a **digital asset-or-nothing** option, the payoff is the underlying asset's price $S_T$ if the option finishes in the money.

- For a Bachelier digital asset-or-nothing $call$, the payout is $S_T$ if $S_T > K$, and 0 otherwise. \
$C_{digital-asset} = S_0\Phi(d) + \sigma\sqrt{T}\phi(d)$ \
where $d = \frac{S_0 - K}{\sigma \sqrt{T}}$.

- For a Bachelier digital asset-or-nothing $put$, the payout is $S_T$ if $S_T < K$, and 0 otherwise. \
$P_{digital-asset} = S_0\Phi(-d) + \sigma\sqrt{T}\phi(-d)$ \
where $d = \frac{S_0 - K}{\sigma \sqrt{T}}$.

 Explain: \
 $V_c=\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty (S_0 - K)e^{-\frac{x^2}{2}}\,dx+\frac{e^{-rT}}{\sqrt{2\pi}}\int_{x^*}^\infty(\sigma\sqrt{T}x)e^{-\frac{x^2}{2}}\,dx=S_0\Phi(d)+\sigma\sqrt{T}\phi(d)$.

In [ ]:
# Bachelier digital asset-or-nothin Call/Put:
def bachelier_digital_asset_call(S0, K, T, sigma):

    d = (S0 - K) / (sigma * np.sqrt(T))
    return S0 * N(d) + sigma * np.sqrt(T) * n(d)

def bachelier_digital_asset_put(S0, K, T, sigma):

    d = (S0 - K) / (sigma * np.sqrt(T))
    return S0 * N(-d) + sigma * np.sqrt(T) * n(-d)


### 3. Black Model 1976 (European):
Given forward price definition: \

$F_t = e^{r(T - t)}S_T$ \

Say the initial discount price $F_0 = S_0e^{rT}$ \

 Let $D(0,T) = e^{-rT}$ to denote the $discount factor$ \

 The prices of the European vanilla options Call/Put:

- Call: $C_{black} = e^{-rT}\left[F_0\Phi(d_1) - K\Phi(d_2)\right]$,
- Put: $P_{black} = e^{-rT}\left[K\Phi(-d_2) - F_0\Phi(-d_1)\right]$. \
\
 where $d_1 = \frac{\log\frac{F_0}{K} + \frac{\sigma^2}{2}T}{\sigma\sqrt{T}}$,  
\
 $d_2 = d_1 - \sigma\sqrt{T}$,

In [ ]:
# Black model vanilla Call/Put:
def black_vanilla_call(F0, K, T, sigma, r):

    if F0 <= 0 or K <= 0 or sigma <= 0 or T <= 0:
        return 0.0
    d1 = (np.log(F0 / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = np.exp(-r * T) * (F0 * N(d1) - K * N(d2))
    return call_price

def black_vanilla_put(F0, K, T, sigma, r):

    if F0 <= 0 or K <= 0 or sigma <= 0 or T <= 0:
        return 0.0
    d1 = (np.log(F0 / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = np.exp(-r * T) * (K * N(-d2) - F0 * N(-d1))
    return put_price

The Black model focus on modeling the forward price $F$ instead of the underlying price. The digital cash-or-nothing for Black model:

- For a Black digital asset-or-nothing $call$, the payout is $S_T$ if $S_T > K$, and 0 otherwise. \
$C_{digital-cash} = e^{-rT}\Phi(d_2)$ \

- For a Bachelier digital asset-or-nothing $put$, the payout is $S_T$ if $S_T < K$, and 0 otherwise. \
$P_{digital-cash} = e^{-rT}\Phi(-d_2)$


In [ ]:
# Black model digital cash-or-nothing:
def black_digital_cash_call(F0, K, T, sigma, r):
    d2 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T)) - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(d2)

def black_digital_cash_put(F0, K, T, sigma, r):
    d2 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T)) - sigma * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(-d2)

The digital asset-or-nothing for Black model:

- For a Black digital asset-or-nothing $call$, the payout is $S_T$ if $S_T > K$, and 0 otherwise. \
$C_{digital-asset} = e^{-rT}\Phi(d_1)$ \

- For a Bachelier digital asset-or-nothing $put$, the payout is $S_T$ if $S_T < K$, and 0 otherwise. \
$P_{digital-asset} = e^{-rT}\Phi(-d_1)$ \

 Derived from: $F_T = F_0 e^{-\frac{\sigma^2 T}{2} + \sigma W_T}$

In [ ]:
# Black model digital asset-or-nothing:
def black_digital_asset_call(F0, K, T, sigma, r):
    d1 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T))
    return F0 * np.exp(-r * T) * norm.cdf(d1)


def black_digital_asset_put(F0, K, T, sigma, r):
    d1 = (np.log(F0/K) + sigma**2/2 * T)/(sigma * np.sqrt(T))
    return F0 * np.exp(-r * T) * norm.cdf(-d1)

### 4. Displaced-Diffusion Model - Shifted Lognormal:
In order to get DD model valnilla Call/Put, we need to parse the $F$, $K$, $σ$, $T$ to the Black Model. \

Black Model: $\quad F_T = F_0 e^{-\frac{\sigma^2 T}{2} + \sigma W_T}$ \

Displaced-Diffusion Model: $\quad F_T = \frac{F_0}{\beta} \exp\left[-\frac{\beta^2 \sigma^2 T}{2} + \beta \sigma W_T\right] - \frac{1 - \beta}{\beta} F_0.$

Displaced-Diffusion: Black($\left( \frac{F_0}{\beta}, \, K + \frac{1 - \beta}{\beta} F_0, \, \sigma \beta, \, T \right)$

In [ ]:
# Displaced-Diffusion model - shifted lognormal:

def displaced_diffusion_vanilla_call(F0, K, T, sigma, beta, r):
  return black_vanilla_call(F0/beta, K + ((1 - beta)/beta) * F0, T, sigma * beta, r)

def displaced_diffusion_vanilla_put(F0, K, T, sigma, beta, r):
  return black_vanilla_put(F0/beta, K + ((1 - beta)/beta) * F0, T, sigma * beta, r)


The digital cash-or-nothing formula for DD Model:
- $C_{DD-digital-cash} = C_{black-cash}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, \sigma\beta, T\right)$ \

- $P_{DD-digital-cash} = P_{black-cash}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, \sigma\beta, T\right)$

In [ ]:
# Displaced-Diffusion digital cash-or-nothing Call/Put:

def displaced_diffusion_digital_cash_call(F0, K, T, sigma, beta, r):
  return black_digital_cash_call(F0/beta, K + ((1 - beta)/beta) * F0, T, sigma * beta, r)

def displaced_diffusion_digital_cash_put(F0, K, T, sigma, beta, r):
  return black_digital_cash_put(F0/beta, K + ((1 - beta)/beta) * F0, T, sigma * beta, r)


The digital asset-or-nothing for DD Model: \

- $C_{DD-digital-asset} = C_{black-asset}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, r, \sigma\beta, T\right)$ \

- $P_{DD-digital-asset} = P_{black-asset}\left(\frac{F_0}{\beta}, K + \frac{1 - \beta}{\beta}F_0, r, \sigma\beta, T\right)$

In [ ]:
# Displaced-Diffusion digital asset-or-nothing Call/Put:

def displaced_diffusion_digital_asset_call(F0, K, T, sigma, beta, r):
    return black_digital_asset_call(F0/beta, K + ((1 - beta)/beta) * F0, T, sigma * beta, r)

def displaced_diffusion_digital_asset_put(F0, K, T, sigma, beta, r):
    return black_digital_asset_put(F0/beta, K + ((1 - beta)/beta) * F0, T, sigma * beta, r)



In [ ]:
S0_low = 5865
S0_high = 5875
K = 5870
T = 7/365
sigma = 0.14
r = 0.05
beta = 0.3
F0_low, F0_high = S0_low * np.exp(r * T), S0_high * np.exp(r * T)

def test_pricing_models(S0_low, S0_high, K, T, sigma, r, beta, F0_low, F0_high):
    scenarios_S = [
        ("S0_low", S0_low),
        ("S0_high", S0_high)
    ]
    scenarios_F = [
        ("F0_low", F0_low),
        ("F0_high", F0_high)
    ]

    bs_models = {
        "Black-Scholes Vanilla": lambda S, option_type: black_scholes_vanilla(S, K, T, r, sigma, option_type),
        "Black-Scholes Digital Cash": lambda S, option_type: black_scholes_digital_cash(S, K, T, r, sigma, option_type),
        "Black-Scholes Digital Asset": lambda S, option_type: black_scholes_digital_asset(S, K, T, r, sigma, option_type),
    }
    bachelier_models = {
        "Bachelier Vanilla": lambda S, option_type: bachelier_vanilla_call(S, K, T, sigma, r) if option_type == 'call' else bachelier_vanilla_put(S, K, T, sigma, r),
        "Bachelier Digital Cash": lambda S, option_type: bachelier_digital_cash_call(S, K, T, sigma) if option_type == 'call' else bachelier_digital_cash_put(S, K, T, sigma),
        "Bachelier Digital Asset": lambda S, option_type: bachelier_digital_asset_call(S, K, T, sigma) if option_type == 'call' else bachelier_digital_asset_put(S, K, T, sigma),
    }
    black_models = {
        "Black Vanilla": lambda F, option_type: black_vanilla_call(F, K, T, sigma, r) if option_type == 'call' else black_vanilla_put(F, K, T, sigma, r),
        "Black Digital Cash": lambda F, option_type: black_digital_cash_call(F, K, T, sigma, r) if option_type == 'call' else black_digital_cash_put(F, K, T, sigma, r),
        "Black Digital Asset": lambda F, option_type: black_digital_asset_call(F, K, T, sigma, r) if option_type == 'call' else black_digital_asset_put(F, K, T, sigma, r),
    }
    DD_models = {
        "Displace-Diffusion Vanilla": lambda F, option_type: displaced_diffusion_vanilla_call(F, K, T, sigma, beta, r) if option_type == 'call' else displaced_diffusion_vanilla_put(F, K, T, sigma, beta, r),
        "Displace-Diffusion Digital Cash": lambda F, option_type: displaced_diffusion_digital_cash_call(F, K, T, sigma, beta, r) if option_type == 'call' else displaced_diffusion_digital_cash_put(F, K, T, sigma, beta, r),
        "Displace-Diffusion Digital Asset": lambda F, option_type: displaced_diffusion_digital_asset_call(F, K, T, sigma, beta, r) if option_type == 'call' else displaced_diffusion_digital_asset_put(F, K, T, sigma, beta, r),
    }

    print("S0-based Models:")
    for model_name, model_func in bs_models.items():
        print(f"{model_name}:")
        for scenario_name, S in scenarios_S:
            call_price = model_func(S, 'call')
            put_price = model_func(S, 'put')
            print(f"  {scenario_name} = {S}: Call Price = {call_price:.3f}, Put Price = {put_price:.3f}")
        print("")
    print("----------------")

    print("S0-based Models (Bachelier):")
    for model_name, model_func in bachelier_models.items():
        print(f"{model_name}:")
        for scenario_name, S in scenarios_S:
            call_price = model_func(S, 'call')
            put_price = model_func(S, 'put')
            print(f"  {scenario_name} = {S}: Call Price = {call_price:.3f}, Put Price = {put_price:.3f}")
        print("")

    print("----------------")

    print("F0-based Models:")
    for model_name, model_func in black_models.items():
        print(f"{model_name}:")
        for scenario_name, F in scenarios_F:
            call_price = model_func(F, 'call')
            put_price = model_func(F, 'put')
            print(f"  {scenario_name} = {F:.3f}: Call Price = {call_price:.3f}, Put Price = {put_price:.3f}")
        print("")

    print("----------------")
    for model_name, model_func in DD_models.items():
        print(f"{model_name}:")
        for scenario_name, F in scenarios_F:
            call_price = model_func(F, 'call')
            put_price = model_func(F, 'put')
            print(f"  {scenario_name} = {F:.3f}: Call Price = {call_price:.3f}, Put Price = {put_price:.3f}")
        print("")

    print("----------------")

test_pricing_models(S0_low, S0_high, K, T, sigma, r, beta, F0_low, F0_high)

test_pricing_models(S0_low, S0_high, K, T, sigma, r, beta, F0_low, F0_high)

S0-based Models:
Black-Scholes Vanilla:
  S0_low = 5865: Call Price = 45.674, Put Price = 45.048
  S0_high = 5875: Call Price = 50.910, Put Price = 40.284

Black-Scholes Digital Cash:
  S0_low = 5865: Call Price = 0.498, Put Price = 0.501
  S0_high = 5875: Call Price = 0.533, Put Price = 0.466

Black-Scholes Digital Asset:
  S0_low = 5865: Call Price = 2968.064, Put Price = 2896.936
  S0_high = 5875: Call Price = 3178.643, Put Price = 2696.357

----------------
S0-based Models (Bachelier):
Bachelier Vanilla:
  S0_low = 5865: Call Price = 0.000, Put Price = 4.995
  S0_high = 5875: Call Price = 4.995, Put Price = 0.000

Bachelier Digital Cash:
  S0_low = 5865: Call Price = 0.000, Put Price = 0.999
  S0_high = 5875: Call Price = 0.999, Put Price = 0.000

Bachelier Digital Asset:
  S0_low = 5865: Call Price = 0.000, Put Price = 5865.000
  S0_high = 5875: Call Price = 5875.000, Put Price = 0.000

----------------
F0-based Models:
Black Vanilla:
  F0_low = 5870.627: Call Price = 45.674, Put 